In [1]:
import tomli
import numpy as np
import pandas as pd
import glob
import math
import csv
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import roc_auc_score

In [2]:
with open("config.toml", "rb") as f:
    config = tomli.load(f)

In [ ]:
train_info = pd.read_csv(config['Train']['InfoFile'])
train_info.head()
# train_info['level'].unique()

,unique_id,player_id,mode,gender,hold racket handed,play years,level,cut_point
0,1,41,1,1,1,1,5,[ 0 61 122 183 244 305 366 428 489 ...
1,2,41,2,1,1,1,5,[ 0 74 149 224 299 374 449 524 599 ...
2,3,41,3,1,1,1,5,[ 0 103 207 311 415 519 623 727 831 ...
3,4,41,4,1,1,1,5,[ 0 101 203 304 406 507 609 710 812 ...
4,5,41,5,1,1,1,5,[ 0 105 211 317 423 529 635 740 846 ...


In [4]:
unique_players = train_info['player_id'].unique()
train_players, test_players = train_test_split(unique_players, test_size=0.2, random_state=42)

# 讀取特徵 CSV 檔（位於 "./tabular_data_train"）
datapath = './tabular_data_train'
datalist = glob.glob(os.path.join(config['Train']['FeaturePath'], "*.csv"))
target_mask = ['gender', 'hold racket handed', 'play years', 'level']

In [ ]:
sample_file = pd.read_csv(datalist[0])
sample_file.head()

,ax_mean,ay_mean,az_mean,gx_mean,gy_mean,gz_mean,ax_var,ay_var,az_var,gx_var,...,a_fft,g_fft,a_psd,g_psd,a_kurt,g_kurt,a_skewn,g_skewn,a_entropy,g_entropy
0,-3210.325581,-1472.639535,769.174419,3888.139535,15000.069767,-1508.011628,2625.734377,2141.530712,2579.672441,22376.099388,...,34235.187587,34235.187587,8.524912e+09,8.524912e+09,7.929079,2.715360,193.153135,63.072307,-0.055732,-0.055732
1,-4266.151163,-1758.790698,433.104651,4868.604651,14022.290698,3694.767442,2716.650412,3104.921440,2033.799252,23827.776754,...,4200.628815,4200.628815,2.855087e+08,2.855087e+08,4.805886,3.387055,116.344046,86.140297,-0.055453,-0.055453
2,-3777.058140,-2363.441860,496.430233,3838.127907,17846.848837,2016.662791,2646.756001,2342.910897,2150.277547,25096.946277,...,8077.753674,8077.753674,2.063152e+08,2.063152e+08,3.828190,2.898664,113.018318,54.248883,-0.056366,-0.056366
3,-3545.058140,-1156.709302,618.046512,3811.674419,14340.662791,3557.930233,2507.900301,2258.403467,1908.365970,22559.762748,...,5265.867665,5265.867665,1.557855e+08,1.557855e+08,2.039432,3.110605,30.070979,67.902479,-0.056986,-0.056986
4,-3078.068966,-793.758621,137.551724,10518.264368,9810.137931,-9929.160920,2518.215706,2126.238452,1811.472957,19047.955661,...,1315.209915,1315.209915,6.176466e+07,6.176466e+07,8.145404,5.094156,205.116213,137.770788,-0.057376,-0.057376


In [6]:
# 根據 test_players 分組資料
x_train = pd.DataFrame()
y_train = pd.DataFrame(columns=target_mask)
x_test = pd.DataFrame()
y_test = pd.DataFrame(columns=target_mask)

for file in datalist:
    unique_id = int(Path(file).stem)
    row = train_info[train_info['unique_id'] == unique_id]
    if row.empty:
        continue
    player_id = row['player_id'].iloc[0]
    data = pd.read_csv(file)
    target = row[target_mask]
    target_repeated = pd.concat([target] * len(data))
    if player_id in train_players:
        x_train = pd.concat([x_train, data], ignore_index=True)
        y_train = pd.concat([y_train, target_repeated], ignore_index=True)
    elif player_id in test_players:
        x_test = pd.concat([x_test, data], ignore_index=True)
        y_test = pd.concat([y_test, target_repeated], ignore_index=True)

In [7]:
# 標準化特徵
scaler = MinMaxScaler()
le = LabelEncoder()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

group_size = 27

In [12]:
x_train.head()

,ax_mean,ay_mean,az_mean,gx_mean,gy_mean,gz_mean,ax_var,ay_var,az_var,gx_var,...,a_fft,g_fft,a_psd,g_psd,a_kurt,g_kurt,a_skewn,g_skewn,a_entropy,g_entropy
0,-3210.325581,-1472.639535,769.174419,3888.139535,15000.069767,-1508.011628,2625.734377,2141.530712,2579.672441,22376.099388,...,34235.187587,34235.187587,8.524912e+09,8.524912e+09,7.929079,2.715360,193.153135,63.072307,-0.055732,-0.055732
1,-4266.151163,-1758.790698,433.104651,4868.604651,14022.290698,3694.767442,2716.650412,3104.921440,2033.799252,23827.776754,...,4200.628815,4200.628815,2.855087e+08,2.855087e+08,4.805886,3.387055,116.344046,86.140297,-0.055453,-0.055453
2,-3777.058140,-2363.441860,496.430233,3838.127907,17846.848837,2016.662791,2646.756001,2342.910897,2150.277547,25096.946277,...,8077.753674,8077.753674,2.063152e+08,2.063152e+08,3.828190,2.898664,113.018318,54.248883,-0.056366,-0.056366
3,-3545.058140,-1156.709302,618.046512,3811.674419,14340.662791,3557.930233,2507.900301,2258.403467,1908.365970,22559.762748,...,5265.867665,5265.867665,1.557855e+08,1.557855e+08,2.039432,3.110605,30.070979,67.902479,-0.056986,-0.056986
4,-3078.068966,-793.758621,137.551724,10518.264368,9810.137931,-9929.160920,2518.215706,2126.238452,1811.472957,19047.955661,...,1315.209915,1315.209915,6.176466e+07,6.176466e+07,8.145404,5.094156,205.116213,137.770788,-0.057376,-0.057376


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import roc_auc_score
import numpy as np

# Define a simple neural network for binary classification
class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

# Define a neural network for multi-class classification
class MultiClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MultiClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        logits = self.model(x)
        return logits
    
    def predict_proba(self, x):
        logits = self.model(x)
        return self.softmax(logits)

# Function to train binary classification model with early stopping (using validation loss)
def train_binary_model(X_train, y_train, X_val, y_val, batch_size=64, epochs=20, lr=0.001, patience=5, min_delta=1e-4,weights=None):
    # Convert data to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1))
    X_val_tensor = torch.FloatTensor(X_val)
    y_val_tensor = torch.FloatTensor(y_val.reshape(-1, 1))
    
    # Create dataset and dataloader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Initialize model
    input_size = X_train.shape[1]
    model = BinaryClassifier(input_size)

    # Loss and optimizer
    if weights != None:
        torch_weight = torch.FloatTensor(weights)
        criterion = nn.BCELoss(weight=torch_weight)
    else:
        criterion = nn.BCELoss()

    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Early stopping variables
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    # Training loop with early stopping (monitoring validation loss)
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        avg_loss = running_loss / len(train_loader)
        
        # Compute validation loss
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor).item()
        
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        # Early stopping check (on validation loss)
        if best_val_loss - val_loss > min_delta:
            best_val_loss = val_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict()
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                if best_model_state is not None:
                    model.load_state_dict(best_model_state)
                break
    
    # Make predictions
    model.eval()
    with torch.no_grad():
        predicted = model(X_val_tensor).numpy().flatten()
    
    # Group predictions
    num_groups = len(predicted) // group_size
    y_pred = []
    for i in range(num_groups):
        group_preds = predicted[i*group_size: (i+1)*group_size]
        if sum(group_preds[:group_size]) / group_size > 0.5:
            y_pred.append(max(group_preds))
        else:
            y_pred.append(min(group_preds))
    
    y_val_agg = [y_val[i*group_size] for i in range(num_groups)]
    
    auc_score = roc_auc_score(y_val_agg, y_pred, average='micro')
    print(f'Binary AUC: {auc_score}')
    return model, auc_score

# Function to train multi-class classification model with early stopping (using validation loss)
def train_multi_model(X_train, y_train, X_val, y_val, batch_size=64, epochs=20, lr=0.001, patience=5, min_delta=1e-4,weights=None):
    # Get number of classes
    num_classes = len(np.unique(y_train))
    
    # Convert data to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.LongTensor(y_train)
    X_val_tensor = torch.FloatTensor(X_val)
    y_val_tensor = torch.LongTensor(y_val)
    
    # Create dataset and dataloader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Initialize model
    input_size = X_train.shape[1]
    model = MultiClassifier(input_size, num_classes)
    
    # Loss and optimizer
    if weights != None:
        torch_weight = torch.FloatTensor(weights)
        criterion = nn.CrossEntropyLoss(weight=torch_weight)
    else:
        criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Early stopping variables
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    # Training loop with early stopping (monitoring validation loss)
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        avg_loss = running_loss / len(train_loader)
        
        # Compute validation loss
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor).item()
        
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        # Early stopping check (on validation loss)
        if best_val_loss - val_loss > min_delta:
            best_val_loss = val_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict()
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                if best_model_state is not None:
                    model.load_state_dict(best_model_state)
                break
    
    # Make predictions
    model.eval()
    with torch.no_grad():
        logits = model(X_val_tensor)
        predicted = torch.softmax(logits, dim=1).numpy()
    
    # Group predictions as in your original code
    num_groups = len(predicted) // group_size
    y_pred = []
    for i in range(num_groups):
        group_pred = predicted[i*group_size: (i+1)*group_size]
        class_sums = [sum([group_pred[k][j] for k in range(group_size)]) for j in range(num_classes)]
        chosen_class = np.argmax(class_sums)
        candidate_probs = [group_pred[k][chosen_class] for k in range(group_size)]
        best_instance = np.argmax(candidate_probs)
        y_pred.append(group_pred[best_instance])
    
    y_val_agg = [y_val[i*group_size] for i in range(num_groups)]
    
    auc_score = roc_auc_score(y_val_agg, y_pred, average='micro', multi_class='ovr')
    print(f'Multi-class AUC: {auc_score}')
    return model, auc_score

In [22]:
# For binary classification (gender)
y_train_le_gender = le.fit_transform(y_train['gender'])
y_test_le_gender = le.transform(y_test['gender'])
binary_model, gender_auc = train_binary_model(X_train_scaled, y_train_le_gender, X_test_scaled, y_test_le_gender,weights=[1600/300])

# For binary classification (hold racket handed)
y_train_le_hold = le.fit_transform(y_train['hold racket handed'])
y_test_le_hold = le.transform(y_test['hold racket handed'])
hold_model, hold_auc = train_binary_model(X_train_scaled, y_train_le_hold, X_test_scaled, y_test_le_hold,weights=[1600/350])

# For multi-class classification (play years)
y_train_le_years = le.fit_transform(y_train['play years'])
y_test_le_years = le.transform(y_test['play years'])
years_model, years_auc = train_multi_model(X_train_scaled, y_train_le_years, X_test_scaled, y_test_le_years,weights=[400,900,700])

# For multi-class classification (level)
y_train_le_level = le.fit_transform(y_train['level'])
y_test_le_level = le.transform(y_test['level'])
level_model, level_auc = train_multi_model(X_train_scaled, y_train_le_level, X_test_scaled, y_test_le_level,weights=[700,200,150,900])


Epoch 1/20, Train Loss: 2.2367, Val Loss: 1.6879
Epoch 2/20, Train Loss: 1.9073, Val Loss: 1.6052
Epoch 3/20, Train Loss: 1.6557, Val Loss: 1.5260
Epoch 4/20, Train Loss: 1.4364, Val Loss: 1.5678
Epoch 5/20, Train Loss: 1.3063, Val Loss: 2.0175
Epoch 6/20, Train Loss: 1.1961, Val Loss: 2.0718
Epoch 7/20, Train Loss: 1.1237, Val Loss: 1.5987
Epoch 8/20, Train Loss: 1.0725, Val Loss: 2.2067
Early stopping at epoch 8
Binary AUC: 0.8620390653523183
Epoch 1/20, Train Loss: 0.8103, Val Loss: 0.1888
Epoch 2/20, Train Loss: 0.1779, Val Loss: 0.1496
Epoch 3/20, Train Loss: 0.1251, Val Loss: 0.0979
Epoch 4/20, Train Loss: 0.1019, Val Loss: 0.0632
Epoch 5/20, Train Loss: 0.0889, Val Loss: 0.0534
Epoch 6/20, Train Loss: 0.0796, Val Loss: 0.0803
Epoch 7/20, Train Loss: 0.0762, Val Loss: 0.0630
Epoch 8/20, Train Loss: 0.0714, Val Loss: 0.0443
Epoch 9/20, Train Loss: 0.0627, Val Loss: 0.0660
Epoch 10/20, Train Loss: 0.0627, Val Loss: 0.0811
Epoch 11/20, Train Loss: 0.0685, Val Loss: 0.0496
Epoch 12/2

In [53]:
def model_binary(X_train, y_train, X_test, y_test):
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train, y_train)
    
    predicted = clf.predict_proba(X_test)
    print(predicted.shape) 
    # 取出正類（index 0）的概率
    predicted = [predicted[i][0] for i in range(len(predicted))]

    
    num_groups = len(predicted) // group_size 
    if sum(predicted[:group_size]) / group_size > 0.5:
        y_pred = [max(predicted[i*group_size: (i+1)*group_size]) for i in range(num_groups)]
    else:
        y_pred = [min(predicted[i*group_size: (i+1)*group_size]) for i in range(num_groups)]
    
    y_pred  = [1 - x for x in y_pred]
    y_test_agg = [y_test[i*group_size] for i in range(num_groups)]
    
    auc_score = roc_auc_score(y_test_agg, y_pred, average='micro')
    print(auc_score)

def model_multiary(X_train, y_train, X_test, y_test):
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train, y_train)
    predicted = clf.predict_proba(X_test)
    num_groups = len(predicted) // group_size
    y_pred = []
    for i in range(num_groups):
        group_pred = predicted[i*group_size: (i+1)*group_size]
        num_classes = len(np.unique(y_train))
        # 對每個類別計算該組內的總機率
        class_sums = [sum([group_pred[k][j] for k in range(group_size)]) for j in range(num_classes)]
        chosen_class = np.argmax(class_sums)
        candidate_probs = [group_pred[k][chosen_class] for k in range(group_size)]
        best_instance = np.argmax(candidate_probs)
        y_pred.append(group_pred[best_instance])
    
    y_test_agg = [y_test[i*group_size] for i in range(num_groups)]
    auc_score = roc_auc_score(y_test_agg, y_pred, average='micro', multi_class='ovr')
    print('Multiary AUC:', auc_score)

In [54]:
# 評分：針對各目標進行模型訓練與評分
y_train_le_gender = le.fit_transform(y_train['gender'])
y_test_le_gender = le.transform(y_test['gender'])
model_binary(X_train_scaled, y_train_le_gender, X_test_scaled, y_test_le_gender)

# y_train_le_hold = le.fit_transform(y_train['hold racket handed'])
# y_test_le_hold = le.transform(y_test['hold racket handed'])
# model_binary(X_train_scaled, y_train_le_hold, X_test_scaled, y_test_le_hold)

# y_train_le_years = le.fit_transform(y_train['play years'])
# y_test_le_years = le.transform(y_test['play years'])
# model_multiary(X_train_scaled, y_train_le_years, X_test_scaled, y_test_le_years)

# y_train_le_level = le.fit_transform(y_train['level'])
# y_test_le_level = le.transform(y_test['level'])
# model_multiary(X_train_scaled, y_train_le_level, X_test_scaled, y_test_le_level)


(10746, 2)
0.7907995618838993
